In [1]:
import os
import cv2
import json
import numpy as np
import pandas
import skimage
import shapefile
import matplotlib.pyplot as plt
import tensorflow as tf
import modules
import sklearn

/home/BenChoi/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/BenChoi/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/BenChoi/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/BenChoi/miniconda3/envs/tf/lib/python3.7/site-

### load data

In [ ]:
# TODO: make TF2 work with GPU (high priority)

# TODO: rescale images to workable size (low priority -- use ben's for now)

# TODO: filter unwanted samples e.g. clouds, outliers, etc. (low priority)

# TODO: center images via mean subtraction (low priority)

# TODO: make temporary train-val-test split (medium priority)

config = modules.run.load_config("cls", tag="w5")
dg = modules.data.DatasetGenerator(config=config)
dataset_kenya = dg.generate_kenya()

In [ ]:
inputs = tf.keras.layers.Input(shape=(config["batch_size"], config["image_size"], config["image_size"], config["n_channels"]))

# TODO: load ResNet, choose loss and optimizer, build out checkpointing and tensorboard.

model = tf.keras.Model()

In [2]:
import modules

config = modules.run.load_config("cls", tag="w5")
dg = modules.data.DatasetGenerator(config=config)
dg._setup("kenya")
sample_filenames = dg.sample("kenya", N=5000, validate=False)
formatted_filenames = ["{}_kenya_resized.jpg".format(sample_filename.split(".")[0].split("_")[1]) for sample_filename, _ in list(zip(*sample_filenames))]

In [5]:
ids = [fname.split("_")[0] for fname in formatted_filenames]
train_df = dg.dataframes['kenya'][dg.dataframes["kenya"]["id"].isin(ids)][["class"]]
train_df["fname"] = formatted_filenames
train_df["fname"] = formatted_filenames

print(train_df[4000:4020])
# train_df = train_df.sample(frac=1)

       class                     fname
index                                 
4544   major   31064_kenya_resized.jpg
4545   major  313941_kenya_resized.jpg
4546   major  258736_kenya_resized.jpg
4547   major   64421_kenya_resized.jpg
4548   major  487718_kenya_resized.jpg
4549   major   61785_kenya_resized.jpg
4550   major  345675_kenya_resized.jpg
4551   major     709_kenya_resized.jpg
4552   major   20393_kenya_resized.jpg
4553   major   47884_kenya_resized.jpg
4555   major  144752_kenya_resized.jpg
4556   major  487386_kenya_resized.jpg
4557   major  224609_kenya_resized.jpg
4558   major  235932_kenya_resized.jpg
4560   major    6821_kenya_resized.jpg
4561   major  493404_kenya_resized.jpg
4563   major  439947_kenya_resized.jpg
4564   major  455755_kenya_resized.jpg
4565   major   43541_kenya_resized.jpg
4566   major   42578_kenya_resized.jpg


In [ ]:
# for el in enumerate(train_df['fname'][1001:1021]):
#     plt.imshow(cv2.imread('local_images/kenya/' + el[1]))
#     plt.show()

In [ ]:
from keras import preprocessing
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import keras

with tf.device('device:GPU:0'):
    
    checkpoints_dir = os.path.join("data", config["base"] + config["tag"], "checkpoints")
    if not os.path.isdir(checkpoints_dir):
        os.makedirs(checkpoints_dir)

    tensorboard_dir = os.path.join("data", config["base"] + config["tag"], "tensorboard")
    if not os.path.isdir(tensorboard_dir):
        os.makedirs(tensorboard_dir)

    tensorboard_callback = keras.callbacks.TensorBoard(
        tensorboard_dir, 
        write_graph=True, 
        write_images=True,
        update_freq=config["batch_size"] * config["tensorboard_freq"]
    )

    checkpoints_callback = keras.callbacks.ModelCheckpoint(
        f"{checkpoints_dir}/model.hdf5",
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        period=1,
    )

    input_layer = keras.layers.Input(
        shape=(
            config["image_size"],
            config["image_size"],
            config["n_channels"])
    )

    if config["balance_classes"]:
        class_weight = sklearn.utils.class_weight.compute_class_weight(
            "balanced", 
            list(range(config["n_classes"])), 
            [config["class_enum"][v] for v in dg.dataframes["kenya"]["class"].values]
        )
    else:
        class_weight = None

    train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input, 
                                             horizontal_flip=True, 
                                             vertical_flip=True,
    #                                          rotation_range=180,
    #                                          brightness_range=[0.0, 2.0],
    #                                          zoom_range=[0.8, 1.0],
    #                                          shear_range=.2,
    #                                          channel_shift_range=.2,
    #                                          width_shift_range=.2,
    #                                          height_shift_range=.2,
    #                                          fill_mode='nearest', 
                                             validation_split=.3)

    train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory='local_images/kenya/',
            subset='training',
            x_col="fname",
            y_col="class",
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical',
            shuffle=True)

    val_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory='local_images/kenya/',
            subset='validation',
            x_col="fname",
            y_col="class",
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical',
            shuffle=True)

    step_size_train = train_generator.n // train_generator.batch_size
    step_size_val = val_generator.n // val_generator.batch_size

    model = VGG16(
        include_top=False,
        weights='imagenet',
        input_tensor=input_layer,
        classes=config["n_classes"]
    )
    
    model.summary(0)

    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(3, activation="softmax")(x)
    model_final = Model(input = model.input, output = predictions)

    for layer in model_final.layers:
            layer.trainable=True
    
    sgd = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
    model_final.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=["categorical_accuracy"])

    model_final.summary()

    step_size_train = train_generator.n // train_generator.batch_size
    step_size_val = val_generator.n // val_generator.batch_size
    
#     model_final.fit_generator(
#             train_generator,
#             steps_per_epoch=step_size_train,
#             epochs=config["n_epochs"],
#             validation_data=val_generator,
#             validation_steps=step_size_val)
    
    model_final.fit_generator(
        train_generator, 
        epochs=50,
        steps_per_epoch=step_size_train,
        callbacks=[tensorboard_callback, checkpoints_callback], 
        validation_data=val_generator, 
        validation_steps=step_size_val,
        class_weight=class_weight
    )

Found 10488 validated image filenames belonging to 3 classes.
Found 4494 validated image filenames belonging to 3 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________

/home/BenChoi/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:103: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/50
327/327 [==============================] - 95s 291ms/step - loss: 1.1349 - categorical_accuracy: 0.4179 - val_loss: 1.0038 - val_categorical_accuracy: 0.4746
Epoch 2/50
327/327 [==============================] - 93s 286ms/step - loss: 1.0018 - categorical_accuracy: 0.4792 - val_loss: 0.9601 - val_categorical_accuracy: 0.5184
Epoch 3/50
327/327 [==============================] - 93s 284ms/step - loss: 0.9361 - categorical_accuracy: 0.5323 - val_loss: 0.9415 - val_categorical_accuracy: 0.5166
Epoch 4/50
327/327 [==============================] - 93s 283ms/step - loss: 0.8723 - categorical_accuracy: 0.5817 - val_loss: 0.8439 - val_categorical_accuracy: 0.5970
Epoch 5/50
327/327 [==============================] - 93s 284ms/step - loss: 0.8359 - categorical_accuracy: 0.6105 - val_loss: 0.8379 - val_categorical_accuracy: 0.6074
Epoch 6/50
327/327 [==============================] - 93s 284ms/step - loss: 0.8117 - categorical_accuracy: 0.6274 - val_loss: 0.8262 - val_categorical_acc

In [ ]:
model_final.evaluate_generator(val_generator, steps=9)

In [ ]:
del model
del model_final